In [22]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# 加载数据
data = pd.read_csv('problem6.csv')

# 检查数据结构
print("原始数据：")
print(data.head())

# 删除日期列，仅保留资产收益数据
returns = data.iloc[:, 1:].pct_change().dropna()  # 假设后两列是资产A和资产B

# 检查数据结构
print("资产收益数据：")
print(returns.head())

# 基本参数
weights = np.array([100, 50])  # 资产A和资产B的权重


原始数据：
   Date          A          B
0     0  92.858572  56.621590
1     1  93.297528  57.480468
2     2  96.196580  58.995037
3     3  95.530679  58.960488
4     4  94.754012  58.677351
资产收益数据：
          A         B
1  0.004727  0.015169
2  0.031073  0.026349
3 -0.006922 -0.000586
4 -0.008130 -0.004802
5  0.010672 -0.012527


# a. Assumptions

In [23]:
# 6a. 计算投资组合的 5% VaR ($ 值)

# 每日收益波动率
sigma_a = returns['A'].std()
sigma_b = returns['B'].std()

# 投资组合波动率
portfolio_volatility = np.sqrt((weights[0] * sigma_a)**2 + (weights[1] * sigma_b)**2)

# 计算 5% VaR 的 $ 值
var_5 = norm.ppf(0.05) * portfolio_volatility
var_5_value = -var_5

print(f"投资组合的 5% VaR ($ 值): ${var_5_value:.2f}")


投资组合的 5% VaR ($ 值): $2.54


# b. Portfolio

In [26]:
# 基本参数
risk_free_rate = 0.0475  # 无风险利率
trading_days = 252  # 一年交易日
strike_a = 100  # 资产A期权行权价
implied_vol_a = 0.20  # 资产A期权隐含波动率
strike_b = 100  # 资产B期权行权价
call_price_b = 6.50  # 资产B期权当前价格
ttm_b = 100 / trading_days  # 资产B期权到期时间（年化）

# c. Using a delta normal approach, calculate the 1 day VaR and ES of the portfolio

In [27]:
# 使用 Delta-Normal 方法计算 1 日 VaR 和 ES

# 1 日 VaR
var_1d = norm.ppf(0.05) * portfolio_volatility

# 1 日 ES
es_1d = portfolio_volatility * norm.pdf(norm.ppf(0.05)) / 0.05

print(f"Delta-Normal 方法的 1 日 VaR: ${-var_1d:.2f}")
print(f"Delta-Normal 方法的 1 日 ES: ${-es_1d:.2f}")


Delta-Normal 方法的 1 日 VaR: $2.54
Delta-Normal 方法的 1 日 ES: $-3.19


# d. Using a Monte Carlo simulation and assuming multivariate normality, calculate the 1 day VaR and ES of the portfolio.

In [28]:
# 使用 Monte Carlo 模拟计算 1 日 VaR 和 ES

n_simulations = 100000  # 模拟次数

# 模拟收益率
simulated_returns = np.random.multivariate_normal(
    mean=returns.mean().values, 
    cov=cov_matrix, 
    size=n_simulations
)

# 模拟投资组合收益
portfolio_returns = simulated_returns @ weights

# 计算 1 日 VaR 和 ES
var_mc = np.percentile(portfolio_returns, 5)
es_mc = portfolio_returns[portfolio_returns <= var_mc].mean()

print(f"Monte Carlo 模拟的 1 日 VaR: ${-var_mc:.2f}")
print(f"Monte Carlo 模拟的 1 日 ES: ${-es_mc:.2f}")


Monte Carlo 模拟的 1 日 VaR: $2.87
Monte Carlo 模拟的 1 日 ES: $3.66


# e. Using the best fit model for each asset (choose between a normal and a tdistribution), calculate VaR and ES for the portfolio.

In [30]:
# 拟合分布并计算 VaR 和 ES

def calculate_var_es(data, alpha=0.05):
    """
    根据最佳拟合分布计算 VaR 和 ES
    """
    # 拟合正态分布
    mu_norm, sigma_norm = norm.fit(data)
    var_norm = norm.ppf(alpha, mu_norm, sigma_norm)
    es_norm = mu_norm - sigma_norm * norm.pdf(norm.ppf(alpha)) / alpha

    # 拟合 t 分布
    df_t, loc_t, scale_t = t.fit(data)
    var_t = t.ppf(alpha, df_t, loc=loc_t, scale=scale_t)
    es_t = t.expect(lambda x: x, args=(df_t,), loc=loc_t, scale=scale_t, lb=-np.inf, ub=var_t) / alpha

    return {'Normal': (-var_norm, -es_norm), 'T-Distribution': (-var_t, -es_t)}

# 对每个资产计算 VaR 和 ES
var_es_a = calculate_var_es(returns['A'])
var_es_b = calculate_var_es(returns['B'])

print("资产A的 VaR 和 ES:")
print(var_es_a)

print("资产B的 VaR 和 ES:")
print(var_es_b)


资产A的 VaR 和 ES:
{'Normal': (0.02173121832769525, 0.02734945653111876), 'T-Distribution': (0.021731526250512525, 0.027349845579694984)}
资产B的 VaR 和 ES:
{'Normal': (0.02246389984153034, 0.028773565911026994), 'T-Distribution': (0.02246374099497776, 0.028773400370169285)}


# f. Compare and contrast the results of a, b, and C. Which would you choose to use and why?

In [31]:
# 比较三种方法的结果

print("比较三种方法的结果:")
print(f"6a 计算的 5% VaR: ${var_5_value:.2f}")
print(f"6c Delta-Normal 方法的 1 日 VaR: ${-var_1d:.2f}, ES: ${-es_1d:.2f}")
print(f"6d Monte Carlo 模拟的 1 日 VaR: ${-var_mc:.2f}, ES: ${-es_mc:.2f}")
print("6e 基于最佳拟合分布的方法已分别计算各资产的 VaR 和 ES。")

# 分析
print("Delta-Normal 方法假设收益率服从正态分布，适用于线性资产组合，而 Monte Carlo 模拟更加灵活，适合处理非线性资产。")
print("拟合 t 分布的方法适合重尾数据，因此在实际中可能更为稳健。")


比较三种方法的结果:
6a 计算的 5% VaR: $2.54
6c Delta-Normal 方法的 1 日 VaR: $2.54, ES: $-3.19
6d Monte Carlo 模拟的 1 日 VaR: $2.87, ES: $3.66
6e 基于最佳拟合分布的方法已分别计算各资产的 VaR 和 ES。
Delta-Normal 方法假设收益率服从正态分布，适用于线性资产组合，而 Monte Carlo 模拟更加灵活，适合处理非线性资产。
拟合 t 分布的方法适合重尾数据，因此在实际中可能更为稳健。
